In [14]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import sys
from statsmodels.regression.linear_model import OLS
from statsmodels.discrete.discrete_model import Logit
from statsmodels.tools import add_constant
import datetime
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm
%matplotlib inline

In [8]:
target_col = "GDI"
predictions = pd.read_csv("./predictions/cnn_%s_predictions_all_epochs.csv" % (target_col),dtype={'videoid':int})

In [9]:
predictions['const'] = 1

In [10]:
train_rmse = []
val_rmse = []
for i in range(100):
    lm = sm.OLS(predictions[predictions['dataset'] == 'train'][target_col].values,
                       predictions[predictions['dataset'] == 'train'][['%s_pred_%s' % (target_col,i),"const"]].values).fit()
    
    predictions['%s_pred_%s_corrected' % (target_col,i)] = lm.predict(predictions[['%s_pred_%s' % (target_col,i),"const"]])
    predictions['error2'] = np.square(predictions[target_col] - predictions['%s_pred_%s_corrected' % (target_col,i)])
    rmses = np.sqrt(predictions.groupby('dataset')['error2'].mean())
    train_rmse.append(rmses.loc['train'])
    val_rmse.append(rmses.loc['validation'])

In [11]:
best_epoch = np.argmin(val_rmse)

In [12]:
print(best_epoch)

73


In [18]:
predictions['%s_pred_corrected' % (target_col)] = predictions['%s_pred_%s_corrected' % (target_col,best_epoch)]

In [19]:
predictions[['%s_pred_corrected' % (target_col),'videoid','side','dataset']].\
    to_csv("./predictions/cnn_%s_predictions_best_epoch.csv" % (target_col),index=False)